In [1]:
import pandas as pd

In [217]:
f = open('./day4/input.txt','r')
lines = f.readlines()

In [218]:
lines[-1]

'iyr:2017 byr:1994\n'

In [260]:
entries= []
row = dict()
for i,line in enumerate(lines):
    
    
    fields = line.replace('\n','').split()
    for field in fields:
        k,v = field.split(':')
        row[k]=v
        
    if (line == '\n') or (i == 0) :
        row = dict()
        entries.append(row)
    


In [261]:
len(lines),len(entries)

(1102, 290)

In [262]:
#lines[243]

In [263]:
pd.DataFrame(entries).columns

Index(['ecl', 'byr', 'iyr', 'eyr', 'pid', 'hcl', 'hgt', 'cid'], dtype='object')

In [264]:
df = pd.DataFrame(entries)

In [265]:
len(df[['hcl', 'ecl', 'hgt', 'byr', 'pid', 'eyr', 'iyr']].dropna())

216

In [266]:
df.iloc[df[['hcl', 'ecl', 'hgt', 'byr', 'pid', 'eyr', 'iyr']].dropna().index]

,ecl,byr,iyr,eyr,pid,hcl,hgt,cid
2,brn,1971,2015,2021,158388040,#18171d,179cm,NaN
3,amb,1936,2014,2030,707057570,#c0946f,186cm,299
4,gry,1939,2013,2020,241741372,#86e981,163cm,203
5,brn,1929,2020,2029,686617364,#341e13,160cm,280
6,hzl,2002,2011,2027,253005469,#623a2f,184cm,NaN
...,...,...,...,...,...,...,...,...
284,oth,1996,2015,2025,322775528,#efcc98,155cm,NaN
286,gry,1929,2011,2029,800222003,#8f8aaa,168cm,NaN
287,hzl,1970,2020,2020,795434985,#623a2f,168cm,209
288,#d624ca,2007,2030,1933,713080083,z,188in,325


In [267]:
(df[['hcl', 'ecl', 'hgt', 'byr', 'pid', 'eyr', 'iyr']].isna().sum(axis=1)==7).sum()

1

In [268]:
(df[['hcl', 'ecl', 'hgt', 'byr', 'pid', 'eyr', 'iyr']].isna().sum(axis=1)==0).sum()

216

In [269]:
df

,ecl,byr,iyr,eyr,pid,hcl,hgt,cid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amb,1943,2014,2028,333051831,NaN,NaN,NaN
2,brn,1971,2015,2021,158388040,#18171d,179cm,NaN
3,amb,1936,2014,2030,707057570,#c0946f,186cm,299
4,gry,1939,2013,2020,241741372,#86e981,163cm,203
...,...,...,...,...,...,...,...,...
285,brn,1960,NaN,2030,764315947,#ceb3a1,183cm,NaN
286,gry,1929,2011,2029,800222003,#8f8aaa,168cm,NaN
287,hzl,1970,2020,2020,795434985,#623a2f,168cm,209
288,#d624ca,2007,2030,1933,713080083,z,188in,325


byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.

In [271]:
#140 too low

In [274]:
df['byr_valid'] = (df['byr'].astype(float).between(1920,2002)) & (df['byr'].str.len() == 4) & (df['byr'].str.isdigit())
df['iyr_valid'] = (df['iyr'].astype(float).between(2010,2020)) & (df['iyr'].str.len() == 4) & (df['iyr'].str.isdigit())
df['eyr_valid'] = (df['eyr'].astype(float).between(2020,2030)) & (df['eyr'].str.len() == 4) & (df['eyr'].str.isdigit())

In [357]:
cms = df['hgt'].fillna('').str.contains('cm')
cmlen = df[cms]['hgt'].str.replace('cm','').astype('float')

ins = df['hgt'].fillna('').str.contains('in')
inchlen = df[ins]['hgt'].str.replace('in','').astype('float')
vin= (inchlen.between(59,76))
vcm = (cmlen.between(150,193)) 

#have to ve very careful here, since the indexs are not the same, can't just use a boolean operation.
df['hgt_valid'] = False
df.loc[vin.index,'hgt_valid'] = vin
df.loc[vcm.index,'hgt_valid'] = vcm

df['hcl_valid'] = (df['hcl'].str.startswith('#')) & (df['hcl'].str.len() == 7) & (df['hcl'].fillna('').str[1:].str.match(r'^[a-z0-9\.]'))
df['ecl_valid'] = df['ecl'].isin(['amb','blu', 'brn', 'gry', 'grn', 'hzl', 'oth'])
df['pid_valid'] = (df['pid'].str.len() ==9) & (df['pid'].str.isdigit())

In [358]:
#df = df.drop('cid',axis=1)

In [359]:
df.columns[df.columns.str.contains('valid')]

Index(['byr_valid', 'iyr_valid', 'eyr_valid', 'hgt_valid', 'hcl_valid',
       'ecl_valid', 'pid_valid'],
      dtype='object')

In [360]:
(df[['hgt_valid', 'byr_valid', 'iyr_valid', 'eyr_valid', 'hcl_valid',
       'ecl_valid', 'pid_valid']].sum(axis=1)==7).sum()

150

In [361]:
df[['hgt_valid', 'byr_valid', 'iyr_valid', 'eyr_valid', 'hcl_valid',
       'ecl_valid', 'pid_valid']].sum()

hgt_valid    220
byr_valid    226
iyr_valid    227
eyr_valid    228
hcl_valid    228
ecl_valid    226
pid_valid    229
dtype: int64

In [362]:
df.loc[(df[['hgt_valid', 'byr_valid', 'iyr_valid', 'eyr_valid', 'hcl_valid',
       'ecl_valid', 'pid_valid']].sum(axis=1)==7)]['iyr'].max()

'2020'

In [346]:
df['myguess'] = (df[['hgt_valid', 'byr_valid', 'iyr_valid', 'eyr_valid', 'hcl_valid',
       'ecl_valid', 'pid_valid']].sum(axis=1)==7)